### Проверка работоспособности и качества пайплайна

Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score
#, roc_curve, scorer
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [2]:
import numpy as np

In [3]:
from sklearn.metrics import precision_score, precision_recall_curve

In [4]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [5]:
X_test.head(3)

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,...,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2,Total_Trans_Amt_Total_Trans_Ct,Total_Revolving_Bal_Total_Trans_Amt_Total_Trans_Ct,Total_Trans_Ct_log,Total_Ct_Chng_Q4_Q1_Total_Trans_Ct
0,44,M,4,Uneducated,Married,$80K - $120K,Silver,25,3,1,...,14335,116,0.758,0.057,0.000032,0.99997,123.577586,16.054691,1.558900,0.006534
1,40,M,3,Uneducated,Divorced,$80K - $120K,Blue,29,3,2,...,3821,81,0.841,0.075,0.000073,0.99993,47.172840,49.392829,1.480342,0.010383
2,34,M,2,Unknown,Married,$60K - $80K,Blue,36,5,2,...,2966,50,1.381,0.833,0.000359,0.99964,59.320000,33.412003,1.364055,0.027620


In [6]:
to_predict_df = X_test.head(1)

In [9]:
to_predict_df.columns.to_list()

['Customer_Age',
 'Gender',
 'Dependent_count',
 'Education_Level',
 'Marital_Status',
 'Income_Category',
 'Card_Category',
 'Months_on_book',
 'Total_Relationship_Count',
 'Months_Inactive_12_mon',
 'Contacts_Count_12_mon',
 'Credit_Limit',
 'Total_Revolving_Bal',
 'Avg_Open_To_Buy',
 'Total_Amt_Chng_Q4_Q1',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1',
 'Avg_Utilization_Ratio',
 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
 'Total_Trans_Amt_Total_Trans_Ct',
 'Total_Revolving_Bal_Total_Trans_Amt_Total_Trans_Ct',
 'Total_Trans_Ct_log',
 'Total_Ct_Chng_Q4_Q1_Total_Trans_Ct']

In [10]:
with open('Churning_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [11]:
predictions = pipeline.predict_proba(X_test)[:, 1]

In [12]:
predictions

array([2.12002875e-05, 2.83550152e-05, 5.26397502e-04, ...,
       2.79601916e-03, 9.57632596e-02, 8.35610746e-02])

In [13]:
def predict_proba_to_int(y_predicted_probs, threshold):
         return (y_predicted_probs > threshold)*1

In [14]:
precision, recall, thresholds = precision_recall_curve(y_test, predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.792680, F-Score=0.892, Precision=0.929, Recall=0.859


In [15]:
# Нашли метки предсказания для лучшего порога
best_predict = predict_proba_to_int(predictions, thresholds[ix])

In [16]:
roc_auc_score(best_predict, y_test)

0.9509257113428607